In [22]:
import pandas as pd
import re
from sklearn.feature_extraction import text
import numpy as np
# from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from itertools import accumulate
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import itertools

# 读取数据
data = pd.read_csv(r"./all_data.tsv", sep='\t',escapechar='\\')
# data2 = pd.read_csv(r"./imdb_train_test.tsv", sep='\t')

def load_stopwords(filepath):
    stopwords_final = []
    #2 ollect words from files. Each word is on one line
    file = open(filepath,"r",encoding="utf-8") #"stopwords-master/baidu.txt"
    for line in file:
        line = line.strip()
        stopwords_final.append(line)
    file.close()
    return stopwords_final

def clean_review(raw_review):
    # 1. 评论是爬虫抓取的，存在一些 html 标签，需要去掉
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. 标点符号只保留 “-” 和 上单引号
    review_text = rex.sub(' ', review_text)
    # 3. 全部变成小写
    review_text = review_text.lower()
    # 4. 分词
    word_list = review_text.split()
    # 5. 词性还原
#     tokens = list(map(lemmatizer.lemmatize, word_list))
#     lemmatized_tokens = list(map(lambda x: lemmatizer.lemmatize(x, "v"), tokens))
    # 6. 去停用词
    meaningful_words = [x for x in word_list if x not in stop_words] #list(filter(lambda x: not x in stop_words))#, lemmatized_tokens
    return meaningful_words

stopwords_filepath = "stop_all.txt"
# stop_words = set(text.ENGLISH_STOP_WORDS)
stop_words = load_stopwords(stopwords_filepath)
rex = re.compile(r'[!"#$%&\()*+,./:;<=>?@\\^_{|}~]+')
# lemmatizer = WordNetLemmatizer()

sentences  = data.sentence.apply(clean_review)

In [23]:
sentences1[0]

['stuff',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'i',
 'insight',
 'guy',
 'i',
 'thought',
 'cool',
 'eighties',
 'make',
 'mind',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'i',
 'remember',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 "mj's",
 'feeling',
 'press',
 'obvious',
 'message',
 'drugs',
 'bad',
 "m'kay",
 'visually',
 'impressive',
 'michael',
 'jackson',
 'remotely',
 'mj',
 'hate',
 'find',
 'boring',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'made',
 'fans',
 'true',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 '20',
 'minutes',
 'excluding',
 'smooth',
 'criminal',
 'sequence',
 'joe',
 'pesci',
 'convincing',
 'psychopathic',
 'powerful',
 'drug',
 'lord',
 'mj',
 'dead',
 'bad',
 'mj',
 'overheard',
 'plans',
 'nah',
 'joe',
 "pesci's

In [24]:
sentences[0]

['stuff',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'i',
 'insight',
 'guy',
 'i',
 'thought',
 'cool',
 'eighties',
 'make',
 'mind',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'i',
 'remember',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 "mj's",
 'feeling',
 'press',
 'obvious',
 'message',
 'drugs',
 'bad',
 "m'kay",
 'visually',
 'impressive',
 'michael',
 'jackson',
 'remotely',
 'mj',
 'hate',
 'find',
 'boring',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'made',
 'fans',
 'true',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 '20',
 'minutes',
 'excluding',
 'smooth',
 'criminal',
 'sequence',
 'joe',
 'pesci',
 'convincing',
 'psychopathic',
 'powerful',
 'drug',
 'lord',
 'mj',
 'dead',
 'bad',
 'mj',
 'overheard',
 'plans',
 'nah',
 'joe',
 "pesci's

In [28]:
len(sentences)

236

In [29]:
cal_len = pd.DataFrame()
cal_len['review_lenght'] = list(map(len, sentences))
print("中位数：", cal_len['review_length'].median())
print("均值数：", cal_len['review_length'].mean())
del cal_len

TypeError: object of type 'float' has no len()

In [ ]:
data['length'] = data['sentence'].apply(lambda x: len(x))
len_df = data.groupby('length').count()
sent_length = len_df.index.tolist()
sent_freq = len_df['sentence'].tolist()

In [ ]:

# 绘制句子长度及出现频数统计图
plt.bar(sent_length, sent_freq)
plt.title("The length of Sentences and the frequency of sentence length")
plt.xlabel("The length of Sentences")
plt.ylabel("The frequency of sentence length")
plt.savefig("./The length of Sentences and the frequency of sentence length.png")
plt.show()
plt.close()

In [ ]:
sent_pentage_list = [(count/sum(sent_freq)) for count in accumulate(sent_freq)]
plt.plot(sent_length, sent_pentage_list)

In [ ]:
# 寻找分位点为quantile的句子长度
quantile = 0.91
#print(list(sent_pentage_list))
for length, per in zip(sent_length, sent_pentage_list):
    if round(per, 2) == quantile:
        index = length
        break
print("\n分位点为%s的句子长度:%d." % (quantile, index))

In [ ]:
# 绘制句子长度累积分布函数图
plt.plot(sent_length, sent_pentage_list)
plt.hlines(quantile, 0, index, colors="c", linestyles="dashed")
plt.vlines(index, 0, quantile, colors="c", linestyles="dashed")
plt.text(0, quantile, str(quantile))
plt.text(index, 0, str(index))
plt.title("The graph of cumulative distribution function of sentence length")
plt.xlabel("The length of sentences")
plt.ylabel("The sentence length accumulates frequency")
plt.savefig("./The graph of cumulative distribution function of sentence length.png")
plt.show()
plt.close()

In [ ]:
np.save('all_sentences.npy',sentences)

In [17]:
embedding_vector_size = 100
w2v_model = Word2Vec(
    sentences=sentences,
    size=embedding_vector_size,sg=1, hs=1,
    min_count=2, window=5, workers=4)
w2v_model.save("word2vec_100dim_sg1_hs1.model")

In [ ]:
embedding_vector_size = 200
w2v_model = Word2Vec(
    sentences=sentences,
    size=embedding_vector_size,sg=1, hs=1,
    min_count=2, window=5, workers=4)
w2v_model.save("word2vec_"+str(embedding_vector_size)+"dim_sg1_hs1.model")

In [18]:



# 取得所有单词
vocab_list = list(w2v_model.wv.vocab.keys())
# 每个词语对应的索引
word_index = {word: index for index, word in enumerate(vocab_list)}
# 序列化
def get_index(sentence):
    global word_index
    sequence = []
    for word in sentence:
        try:
            sequence.append(word_index[word])
        except KeyError:
            pass
    return sequence
X_data = list(map(get_index, sentences))

# 截长补短
# max_len 根据中位数和平均值得来的
maxlen = 190
X_pad = pad_sequences(X_data, maxlen=maxlen)
# 取得标签
Y = data.label.values+data2.label.values
np.save("Y.npy",Y)
# 划分数据集
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad,
    Y,
    test_size=0.2,
    random_state=42)
np.save("X_train.npy",X_train)
np.save("X_test.npy",X_test)
np.save("Y_train.npy",Y_train)
np.save("Y_test.npy",Y_test)

In [19]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense

# 让 Keras 的 Embedding 层使用训练好的Word2Vec权重
embedding_matrix = w2v_model.wv.vectors

model = Sequential()
model.add(Embedding(
    input_dim=embedding_matrix.shape[0],
    output_dim=embedding_matrix.shape[1],
    input_length=maxlen,
    weights=[embedding_matrix],
    trainable=False))
model.add(Bidirectional(LSTM(128, recurrent_dropout=0.1)))
model.add(Dropout(0.25))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    x=X_train,
    y=Y_train,
    validation_data=(X_test, Y_test),
    batch_size=1024,
    epochs=20
) 
# model_save_path= './labelled_train_test_model.h5'
model_save_path= './imdb_train_test_model_BILSTM1.h5'
model.save(model_save_path)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 15s 729us/step - loss: 0.7161 - acc: 0.5023 - val_loss: 0.6897 - val_acc: 0.5322
Epoch 2/20
 5120/20000 [======>.......................] - ETA: 8s - loss: 0.7102 - acc: 0.5066

KeyboardInterrupt: 

In [ ]:
def plot_loss_accuracy(his):
    loss = his.history['loss']
    val_loss = his.history['val_loss']
    acc = his.history['acc']
    val_acc = his.history['val_acc']

    # make a figure
    fig = plt.figure(figsize=(12,6))
    # subplot loss
    ax1 = fig.add_subplot(121)
    ax1.plot(loss,label='train_loss')
    ax1.plot(val_loss,label='val_loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss on Training and Validation Data')
    ax1.legend()
    # subplot acc
    ax2 = fig.add_subplot(122)
    ax2.plot(acc,label='train_acc')
    ax2.plot(val_acc,label='val_acc')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Accuracy  on Training and Validation Data')
    ax2.legend()
    plt.tight_layout()

In [ ]:
plot_loss_accuracy(history)

In [ ]:


def plot_confusion_matrix(cm, classes,title="Confusion Matrix",cmap=plt.cm.Blues):
    """
    The function prints and plots the confusion matrix
    """
    plt.imshow(cm,interpolation="nearest",cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=0)
    plt.yticks(tick_marks,classes)
    
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j, i, cm[i,j],horizontalalignment = 'center',
        color="white" if cm[i,j]>thresh else"black")
        
    plt.tight_layout()
    plt.ylabel("True label")

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict_classes(X_test)
cnf_matrix = confusion_matrix(Y_test, y_pred)
accuracy = (cnf_matrix[0,0]+cnf_matrix[1,1])/(cnf_matrix[0,0]+cnf_matrix[0,1]+cnf_matrix[1,0]+cnf_matrix[1,1])
precision = cnf_matrix[1,1]/(cnf_matrix[1,1]+cnf_matrix[0,1])
recall = cnf_matrix[1,1]/(cnf_matrix[1,1]+cnf_matrix[1,0])
specificity = cnf_matrix[0,0]/(cnf_matrix[0,0]+cnf_matrix[0,1])
f1 = 2*precision*recall/(precision+recall)
print("accuracy metric in the testing dataset: ", accuracy)
print("precision metric in the testing dataset: ", precision)
print("recall metric in the testing dataset: ", recall)
print("specificity metric in the testing dataset: ", specificity)
print("F1 score metric in the testing dataset: ", f1)
class_names=[0,1]
plt.figure
plot_confusion_matrix(cnf_matrix,classes=class_names, title="Confusion matrix")
plt.show()

In [ ]:
# 序列化
def get_predict_index(sentence,word_index):
    sequence = []
    for word in sentence:
        try:
            sequence.append(word_index[word])
        except KeyError:
            pass
    return sequence

In [ ]:
def predict(text,word2_vecpath,model):
    stopwords = load_stopwords("stop_all.txt")
    sentence = clean_review(text)
    w2v = Word2Vec.load(word2_vecpath)
    # 取得所有单词
    vocab_list = list(w2v_model.wv.vocab.keys())
    # 每个词语对应的索引
    word_index = {word: index for index, word in enumerate(vocab_list)}
    seq_sentence = get_predict_index(sentence,word_index)
    maxlen = 90
    X_pad = pad_sequences(sentence, maxlen=maxlen)
    y_pred = model.predict_classes(X_pad)
    t_prob = model.predict_proba(X_pad)
    print(y_pred,t_prob)